In [8]:
!pip install ultralytics
!pip install simple-online-realtime-tracking

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 30.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110459 sha256=5bb6702dd7ae055a5ed2523e3b4945ffe6b051a2207a8a4414fc92ee3a1205ca
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
  Created wheel for lap: filename=lap-0.4.0-cp310-cp310-linux_x86_64.whl size=1628947 sha256=b84083630e9235e91a4794c5843cedfbbd9ece98a1d19c77741c8ad8937686f4
  Stored in directory: /root/.cache/pip/wheels/00/42/2e/9dfe19270eea279d79e84767ff0d7b8082c3bf776cad00e83d
Successfully built filterpy lap


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf

# Carregar o modelo de temperatura treinado
temperature_model = tf.keras.models.load_model('/content/drive/MyDrive/temperature_digits/temperature_model.h5')


In [12]:
# Function to preprocess the digit image
def preprocess_digit(digit):
    digit_resized = cv2.resize(digit, (16, 21))
    digit_normalized = digit_resized.astype('float32') / 255.0
    digit_ready = np.expand_dims(np.expand_dims(digit_normalized, axis=0), axis=-1)
    return digit_ready

# Function to recognize the digit using the CNN model
def recognize_digit(digit):
    preprocessed_digit = preprocess_digit(digit)
    prediction = temperature_model.predict(preprocessed_digit, verbose=0)
    predicted_class = np.argmax(prediction)

    class_mapping = {
        0: '0', 1: '1', 2: '2', 3: '3', 4: '4',
        5: '5', 6: '6', 7: '7', 8: '8', 9: '9',
        10: 'minus', 11: 'nothing'
    }

    return class_mapping.get(predicted_class, 'unknown')

# Huber loss calculation for robust estimation of temperature
def huber_loss(residual, delta=1.0):
    if abs(residual) <= delta:
        return 0.5 * residual ** 2
    else:
        return delta * (abs(residual) - 0.5 * delta)

# Function to calculate robust mean using Huber loss-based weighting
def robust_temperature_mean(temperatures, delta=1.0):
    mean_temp = np.mean(temperatures)
    weighted_sum = 0
    total_weight = 0
    for temp in temperatures:
        residual = temp - mean_temp
        loss = huber_loss(residual, delta)
        weight = 1 / (1 + loss)
        weighted_sum += temp * weight
        total_weight += weight

    return weighted_sum / total_weight

# Function to extract temperature from a rectangular region
def extract_temperature(image, x1, y1, x2, y2):
    roi = image[y1:y2, x1:x2]

    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

    digit1 = gray_roi[:, :16]
    digit2 = gray_roi[:, 16:32]
    digit3 = gray_roi[:, 48:64]

    recognized_digit1 = recognize_digit(digit1)
    recognized_digit2 = recognize_digit(digit2)
    recognized_digit3 = recognize_digit(digit3)

    if recognized_digit1 == 'minus':
        recognized_number = f"-{recognized_digit2}.{recognized_digit3}"
    else:
        recognized_number = f"{recognized_digit1}{recognized_digit2}.{recognized_digit3}"

    #print(f"Recognized temperature: {recognized_number}")

    try:
        return float(recognized_number)
    except ValueError:
        return None

# Function to process the thermal image and compute the final robust temperature
def process_thermal_image(image_path, x1, y1, x2, y2):
    img = image_path

    if img is None:
        #print(f"Could not read the image: {image_path}")
        return

    # Extract high and low reference temperatures
    temp_high = extract_temperature(img, 510, 67, 575, 88)
    temp_low = extract_temperature(img, 510, 403, 575, 424)

    if temp_high is not None and temp_low is not None:
        #print(f"High temperature: {temp_high}°C")
        #print(f"Low temperature: {temp_low}°C")

        temp_range = temp_high - temp_low

        def get_pixel_temp(pixel_value):
            normalized = pixel_value / 255.0
            return temp_low + (normalized * temp_range)

        # Extract pixel temperatures in the defined rectangle
        pixel_temps = []
        for y in range(y1, y2):
            for x in range(x1, x2):
                pixel_value = img[y, x, 0]
                pixel_temp = get_pixel_temp(pixel_value)
                pixel_temps.append(pixel_temp)

        # Compute the robust mean temperature using Huber loss-based estimation
        final_temp = robust_temperature_mean(pixel_temps, delta=1.0)
        return final_temp
    else:
        #print("Could not extract temperatures from the image.")
        return None

In [9]:
import cv2
import numpy as np
from ultralytics import YOLO
from sort import Sort
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

# Paths
model_path = '/content/drive/MyDrive/yolo/best (2).pt'
input_video_path = '/content/drive/MyDrive/yolo/processed_video (4).mp4'
output_video_path = '/content/output_video.mp4'

In [10]:
# Carregar o modelo YOLOv8
model = YOLO(model_path)  # Substitua por seu modelo YOLOv8, como 'yolov8s.pt', 'yolov8m.pt', etc.

# Inicializar o SORT tracker
tracker = Sort()

In [14]:
import cv2
import numpy as np
from sort import Sort  # Certifique-se de que o SORT está corretamente instalado e importado

# Inicialize rastreadores separados para cabeças e olhos
tracker_heads = Sort()
tracker_eyes = Sort()

# Inicialize conjuntos para armazenar IDs únicos de cabeças e olhos
head_ids = set()
eye_ids = set()

# Abra o arquivo de vídeo
cap = cv2.VideoCapture(input_video_path)  # Substitua pelo caminho do seu vídeo

# Defina a saída do vídeo
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, 25.0, (int(cap.get(3)), int(cap.get(4))))

# Defina os IDs das classes conforme seu modelo YOLOv8
CLASS_HEAD = 0  # Substitua pelo ID real da classe "cabeça" no seu modelo
CLASS_EYE = 1   # Substitua pelo ID real da classe "olho" no seu modelo

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Use o modelo YOLOv8 para fazer inferências no frame
    results = model(frame)

    # Obtenha as caixas delimitadoras, confidências e IDs das classes
    detections = []
    for result in results[0].boxes:
        xyxy = result.xyxy.cpu().numpy().astype(int).flatten()  # Coordenadas [x1, y1, x2, y2]
        conf = result.conf.cpu().numpy().flatten()[0]           # Confiança
        cls = int(result.cls.cpu().numpy().flatten()[0])        # ID da classe
        if conf > 0.2:  # Filtrar por confiança (ajuste conforme necessário)
            detections.append([*xyxy, conf, cls])  # Inclui o ID da classe

    # Converter detecções para um array NumPy
    dets = np.array(detections)

    # Assegurar que 'dets' seja 2D
    if dets.ndim == 1:
        if dets.size == 0:
            dets = dets.reshape(0, 6)  # Sem detecções
        else:
            dets = dets.reshape(1, 6)  # Uma única detecção

    # Inicialize arrays para detecções de cabeças e olhos separadamente
    dets_heads = dets[dets[:, -1] == CLASS_HEAD]  # [x1, y1, x2, y2, conf, cls]
    dets_eyes = dets[dets[:, -1] == CLASS_EYE]    # [x1, y1, x2, y2, conf, cls]

    # Aplicar os rastreadores SORT para cabeças e olhos separadamente
    tracked_heads = tracker_heads.update(dets_heads)
    tracked_eyes = tracker_eyes.update(dets_eyes)

    # Processar cabeças rastreadas
    for obj in tracked_heads:
        x1, y1, x2, y2, obj_id = obj.astype(int)[:5]
        head_ids.add(obj_id)  # Adiciona o ID ao conjunto de cabeças
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 3)
        cv2.putText(frame, f'Cabeça ID {obj_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

    for obj in tracked_eyes:
        x1, y1, x2, y2, obj_id = obj.astype(int)[:5]
        eye_ids.add(obj_id)  # Adiciona o ID ao conjunto de olhos
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 3)

        eye_temp = process_thermal_image(frame, x1, y1, x2, y2)
        cv2.putText(frame, f'Olho ID {obj_id}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 0, 0), 2)


    # Escreve o frame processado no vídeo de saída
    out.write(frame)

    # Opcional: Exibir o frame
    # cv2.imshow('frame', frame)
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break

# Após o processamento, obtenha as contagens
total_cabecas = len(head_ids)
total_olhos = len(eye_ids)

print(f'Total de cabeças detectadas: {total_cabecas}')
print(f'Total de olhos detectados: {total_olhos}')

# Limpeza
cap.release()
out.release()
cv2.destroyAllWindows()



0: 512x640 (no detections), 8.7ms
Speed: 2.5ms preprocess, 8.7ms inference, 0.6ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 (no detections), 8.4ms
Speed: 1.9ms preprocess, 8.4ms inference, 0.6ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 (no detections), 8.3ms
Speed: 2.5ms preprocess, 8.3ms inference, 0.6ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 (no detections), 7.9ms
Speed: 2.2ms preprocess, 7.9ms inference, 0.6ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 (no detections), 8.2ms
Speed: 1.8ms preprocess, 8.2ms inference, 0.6ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 (no detections), 7.8ms
Speed: 1.6ms preprocess, 7.8ms inference, 0.5ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 (no detections), 7.8ms
Speed: 1.3ms preprocess, 7.8ms inference, 0.6ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 (no detections), 7.7ms
Speed: 1.8ms preprocess, 7.7ms inference, 0.5ms 